In [9]:
import sys
import subprocess
import warnings

# --- INSTALACIÓN ROBUSTA DE DEPENDENCIAS ---
print(">>> Verificando e instalando librerías gráficas...")
packages = ["plotly", "pandas", "kaleido", "fpdf"] 
for package in packages:
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "--quiet", "--no-cache-dir"])
    except:
        pass

warnings.filterwarnings("ignore", category=DeprecationWarning)

# Reiniciamos imports después de instalar
import plotly.express as px
import plotly.io as pio
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, avg, count, year, month, desc, expr
import os
from fpdf import FPDF
import datetime

>>> Verificando e instalando librerías gráficas...


In [10]:
# --- CONFIGURACIÓN ---
pio.renderers.default = "iframe"

DB_HOST = "sql_server"
DB_PORT = "1433"
DB_USER = "sa"
DB_PASS = "PasswordFuerte123!" 
DB_DW = "DW_AlquilerHabitacion" 
DRIVER_PKG = "com.microsoft.sqlserver:mssql-jdbc:11.2.0.jre8"
DRIVER_CLASS = "com.microsoft.sqlserver.jdbc.SQLServerDriver"

print("\n--- INICIANDO SESIÓN DE ANÁLISIS OLAP ---")

spark = SparkSession.builder \
    .appName("OLAP-Dashboard") \
    .master("local[*]") \
    .config("spark.jars.packages", DRIVER_PKG) \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

# Función de lectura optimizada
def leer_dw(tabla):
    url = f"jdbc:sqlserver://{DB_HOST}:{DB_PORT};databaseName={DB_DW};encrypt=true;trustServerCertificate=true;"
    return spark.read.format("jdbc") \
        .option("url", url).option("dbtable", tabla) \
        .option("user", DB_USER).option("password", DB_PASS) \
        .option("driver", DRIVER_CLASS).load()


--- INICIANDO SESIÓN DE ANÁLISIS OLAP ---


In [4]:
print(">>> Construyendo el Cubo OLAP Virtual...")

# Cargamos las tablas
df_hechos = leer_dw("FACT_PAGO")
df_dim_cli = leer_dw("DIM_CLIENTE")
df_dim_hab = leer_dw("DIM_HABITACION")
df_dim_tie = leer_dw("DIM_TIEMPO")
df_dim_pro = leer_dw("DIM_PROPIETARIO")

df_cubo = df_hechos.alias("f") \
    .join(df_dim_cli.alias("c"), col("f.idClienteDW") == col("c.idClienteDW")) \
    .join(df_dim_hab.alias("h"), col("f.idHabitacionDW") == col("h.idHabitacionDW")) \
    .join(df_dim_tie.alias("t"), col("f.idTiempoDW") == col("t.idTiempoDW")) \
    .join(df_dim_pro.alias("p"), col("f.idPropietarioDW") == col("p.idPropietarioDW")) \
    .select(
        col("t.anio"), col("t.mes"), col("t.fecha"),
        col("h.tipoHabitacion"), col("h.numeroHabitacion"),
        col("c.nombreCli"), col("c.dniCli"),
        col("p.nombrePropietario"),
        col("f.montoPago"), col("f.metodoPago")
    )

df_cubo.cache()
print(f"   ✅ Cubo cargado con {df_cubo.count()} registros.")


>>> Construyendo el Cubo OLAP Virtual...
   ✅ Cubo cargado con 10796 registros.


In [4]:
# Agregación Jerárquica
print("\n>>> Reporte Jerárquico (Roll-up): Año -> Mes")

# Spark .rollup() nos permite calcular subtotales automáticos
df_rollup = df_cubo.rollup("anio", "mes").agg(sum("montoPago").alias("Total_Ingresos")) \
    .orderBy("anio", "mes") \
    .filter(col("anio").isNotNull()) # Filtramos el total global "null" para el gráfico

# Convertimos a Pandas para visualizar
pdf_rollup = df_rollup.toPandas()

# Limpieza para gráfico: Rellenar nulos de mes con "Total Anual"
pdf_rollup['mes'] = pdf_rollup['mes'].fillna(' TOTAL ANUAL')

# Visualización: Tabla Pivote
print("   -> Mostrando Tabla de Resumen:")
display(pdf_rollup.head(15)) # En Jupyter esto muestra una tabla bonita


>>> Reporte Jerárquico (Roll-up): Año -> Mes
   -> Mostrando Tabla de Resumen:


,anio,mes,Total_Ingresos
0,2024,TOTAL ANUAL,3229400.00
1,2024,1.0,68250.00
2,2024,2.0,114850.00
3,2024,3.0,182000.00
4,2024,4.0,225200.00
5,2024,5.0,261900.00
6,2024,6.0,278050.00
7,2024,7.0,322850.00
8,2024,8.0,339600.00
9,2024,9.0,331450.00


In [5]:
# OPERACIÓN OLAP Filtros y Cortes
print("\n>>> Dashboard: Rentabilidad por Habitación (Slice: Año Actual)")

# SLICE: Filtramos solo el año más reciente
anio_analisis = 2024
df_slice = df_cubo.filter(col("anio") == anio_analisis)

# Agrupamos por Tipo de Habitación
df_kpi_hab = df_slice.groupBy("tipoHabitacion").agg(
    sum("montoPago").alias("Ingresos"),
    count("montoPago").alias("Transacciones"),
    avg("montoPago").alias("Ticket_Promedio")
).orderBy(col("Ingresos").desc())

pdf_kpi = df_kpi_hab.toPandas()

# Gráfico de Barras Interactivo
fig_bar = px.bar(pdf_kpi, x="tipoHabitacion", y="Ingresos", 
                 color="Ingresos", title=f"Ingresos por Tipo de Habitación ({anio_analisis})",
                 text_auto='.2s', color_continuous_scale='Viridis')
fig_bar.show()



>>> Dashboard: Rentabilidad por Habitación (Slice: Año Actual)


In [6]:
# ANÁLISIS DE PARETO 
print("\n>>> Dashboard: Top Clientes (Pareto)")

df_pareto = df_cubo.groupBy("nombreCli").agg(sum("montoPago").alias("Total_LTV")) \
    .orderBy(col("Total_LTV").desc()).limit(10)

pdf_pareto = df_pareto.toPandas()

fig_pareto = px.funnel(pdf_pareto, y='nombreCli', x='Total_LTV', 
                       title="Top 10 Clientes por Valor de Vida (LTV)")
fig_pareto.show()


>>> Dashboard: Top Clientes (Pareto)


In [7]:
# EVOLUCIÓN TEMPORAL (TIME SERIES)
print("\n>>> Dashboard: Tendencia de Ingresos")

df_trend = df_cubo.groupBy("anio", "mes").agg(sum("montoPago").alias("Ventas")) \
    .orderBy("anio", "mes")

pdf_trend = df_trend.toPandas()

pdf_trend['Periodo'] = pdf_trend['anio'].astype(str) + '-' + pdf_trend['mes'].astype(str).str.zfill(2)

fig_line = px.line(pdf_trend, x='Periodo', y='Ventas', markers=True,
                   title="Evolución Mensual de Ingresos")
fig_line.update_layout(hovermode="x unified")
fig_line.show()

print("\nGeneración de Dashboard Completada.")


>>> Dashboard: Tendencia de Ingresos



Generación de Dashboard Completada.


In [11]:
print("\n>>> Generando PDF 'Bonito'...")

# Crear carpeta para temporales
if not os.path.exists("reporte_temp"):
    os.makedirs("reporte_temp")

try:
    print("   -> Exportando gráficos a imágenes estáticas...")
    
    try:
        import kaleido
    except ImportError:
        print("   ⚠️ Kaleido no detectado en el entorno actual. Intentando instalar...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "kaleido", "--quiet"])
    
    try:
        fig_bar.write_image("reporte_temp/rentabilidad.png", scale=2)
        fig_pareto.write_image("reporte_temp/pareto.png", scale=2)
        if not pdf_trend.empty:
            fig_line.write_image("reporte_temp/tendencia.png", scale=2)
    except Exception as e:
        print(f"   ⚠️ Alerta al exportar imágenes (el PDF podría salir sin gráficos): {e}")

    print("   -> Maquetando PDF...")
    
    class PDF(FPDF):
        def header(self):
            self.set_font('Arial', 'B', 15)
            self.cell(0, 10, 'Reporte Ejecutivo de Inteligencia de Negocios', 0, 1, 'C')
            self.ln(5)

        def footer(self):
            self.set_y(-15)
            self.set_font('Arial', 'I', 8)
            self.cell(0, 10, f'Pagina {self.page_no()}', 0, 0, 'C')

    pdf = PDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    # Introducción
    pdf.cell(0, 10, f"Fecha: {datetime.datetime.now().strftime('%d/%m/%Y')}", 0, 1)
    pdf.ln(5)
    pdf.multi_cell(0, 10, "Este reporte presenta los indicadores clave de rendimiento (KPIs) extraidos del Data Warehouse. Se incluyen analisis de rentabilidad por habitacion, segmentacion de clientes y proyeccion temporal.")
    pdf.ln(10)

    # 1. Rentabilidad
    pdf.set_font("Arial", 'B', 14)
    pdf.cell(0, 10, "1. Rentabilidad por Habitacion", 0, 1)
    if os.path.exists("reporte_temp/rentabilidad.png"):
        pdf.image("reporte_temp/rentabilidad.png", x=10, w=190)
    else:
        pdf.cell(0, 10, "[Grafico no disponible]", 0, 1)
    pdf.ln(5)
    
    # 2. Pareto
    pdf.add_page()
    pdf.set_font("Arial", 'B', 14)
    pdf.cell(0, 10, "2. Top Clientes VIP", 0, 1)
    if os.path.exists("reporte_temp/pareto.png"):
        pdf.image("reporte_temp/pareto.png", x=10, w=190)
    pdf.ln(5)

    # 3. Tendencia
    pdf.cell(0, 10, "3. Evolucion de Ingresos", 0, 1)
    if os.path.exists("reporte_temp/tendencia.png"):
        pdf.image("reporte_temp/tendencia.png", x=10, w=190)

    # Guardar
    output_file = "Reporte_Gerencial_Alquileres.pdf"
    pdf.output(output_file)
    
    print(f"   ✅ ¡PDF Generado! Archivo guardado como: {output_file}")

except Exception as e:
    print(f"   ❌ Error generando PDF: {e}")

print("\n✅ Proceso Finalizado.")


>>> Generando PDF 'Bonito'...
   -> Exportando gráficos a imágenes estáticas...
   -> Maquetando PDF...
   ✅ ¡PDF Generado! Archivo guardado como: Reporte_Gerencial_Alquileres.pdf

✅ Proceso Finalizado.
